In [1]:
import matplotlib.pyplot as plt
import pnstyler.styler as styler
from data_preprocess import trajectory_to_ase_atoms, preprocess_asedb
import ml_confs

/home/pnovelli/miniforge3/envs/schnetpack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/pnovelli/miniforge3/envs/schnetpack/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
configs = {
    'topology_path': '/media/data/pnovelli/md_datasets/chignolin-unbiased/CLN025-0-protein.pdb',
    'trajectory_path': '/media/data/pnovelli/md_datasets/chignolin-unbiased/CLN025-0-protein-ALL.dcd',
    'selection': 'backbone',
    'data_path': '/home/pnovelli/code/dp_examples/chignolin/schnetpack/data'
}
configs = ml_confs.from_dict(configs)

In [3]:
#trajectory_to_ase_atoms(configs)

In [4]:
cutoff = 10.
#preprocess_asedb(cutoff, '/home/pnovelli/code/dp_examples/chignolin/schnetpack/data/CLN025-0-protein_backbone.db')

In [5]:
from pathlib import Path
import schnetpack
import logging 
from input_pipeline import TimeLaggedSampler
from model import GraphDPNet
import torch
import lightning

db_path = Path('/home/pnovelli/code/dp_examples/chignolin/schnetpack/data/CLN025-0-protein_backbone.db')

In [6]:
def get_dataset(db_path, cutoff):
    data_path = db_path.parent
    db_name = db_path.name.split(".")[0]
    cache_path = str(data_path / f"__cache-{db_name}__")
    nb_list_transform = schnetpack.transform.CachedNeighborList(
        cache_path,
        schnetpack.transform.MatScipyNeighborList(cutoff=cutoff),
        keep_cache=True,
    )
    in_transforms = [schnetpack.transform.CastTo32(), nb_list_transform]
    return schnetpack.data.ASEAtomsData(str(db_path), transforms=in_transforms)

In [7]:
dataset = get_dataset(db_path, cutoff)

In [11]:
n_atoms = dataset[0][schnetpack.properties.n_atoms].item()
nn_configs = {
    "n_final_features": 16,
    "n_atom_basis": 64,
    "n_interactions": 3,
    "n_rbf": 20,
    "cutoff": cutoff
}
nn_configs = ml_confs.from_dict(nn_configs)
model = GraphDPNet(nn_configs, n_atoms, torch.optim.Adam, use_relaxed_loss = True, metric_deformation_loss_coefficient=1.0,  optimizer_kwargs= {'lr': 1e-3})

In [22]:
#train_logger = lightning.pytorch.loggers.WandbLogger(project='GraphDPNet-chignolin', entity='csml')

lr_finder_cb = lightning.pytorch.callbacks.LearningRateFinder()

ckpt_path = db_path.parent.parent / 'ckpt' / str(train_logger.experiment.name)
ckpt_cb = lightning.pytorch.callbacks.ModelCheckpoint()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pietronvll (csml). Use `wandb login --relogin` to force relogin


In [23]:
trainer = lightning.Trainer(
    accelerator='gpu', 
    #logger=train_logger, 
    max_epochs=2, 
    callbacks=[lr_finder_cb, ckpt_cb],
)

INFO: Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO:lightning.pytorch.utilities.rank_zero:Trainer will use only 1 of 8 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=8)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU av

In [24]:
trainer.fit(model, dataloader)

/home/pnovelli/miniforge3/envs/schnetpack/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:43: attribute 'configs' removed from hparams because it cannot be pickled
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


AttributeError: 'GraphDPNet' object has no attribute '_optimizer'

In [ ]:
#Data loading
dataset = get_dataset(db_path, cutoff)
batch_sampler = TimeLaggedSampler(dataset, batch_size = configs.training.batch_size, lagtime = configs.data.lagtime, shuffle = configs.training.shuffle)
dataloader = schnetpack.data.AtomsLoader(dataset, batch_sampler=batch_sampler, num_workers=20, persistent_workers=True)


#Optimizer
optimizer = torch.optim.Adam
#Model
n_atoms = dataset[0][schnetpack.properties.n_atoms].item()
model = GraphDPNet(configs, n_atoms, optimizer, use_relaxed_loss = True, metric_deformation_loss_coefficient=1.0,  optimizer_kargs= {'lr': 1e-3})
logger = lightning.loggers.WandbLogger(project=configs.wandb.project_name, config=configs.to_dict(), entity=configs.wandb.entity)



ckpt_path = os.path.join(ckpt_path, str(logger.experiment.name))
ckpt_callback = pl.callbacks.ModelCheckpoint(
    monitor='val/P',
    dirpath = ckpt_path,
    mode = 'max', 
    save_top_k = 5, 
    save_last = True
)

trainer = lightning.Trainer(
    accelerator='gpu', 
    logger=logger, 
    max_epochs=configs.training.epochs, 
    log_every_n_steps=configs.training.log_every,
    callbacks=[ckpt_callback],
    limit_val_batches = 10,
    val_check_interval = configs.training.log_every
    )
trainer.fit(model, dataloader, val_dataloaders=dataloader) #Using the same data for tranining and validation is not kosher, but it's a hack for now.